In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import numpy as np
import mojimoji 

import sys
sys.path.append("/net/nfs2/export/home/ohno/CR_pytorch/class")
import CharToIndex

In [3]:
origin_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki/origin.npy"
origin = np.load(origin_file_path,allow_pickle=True)

In [4]:
corpus = origin[0]

In [6]:
chars_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki/all_chars_3565.npy"
tokens = CharToIndex.CharToIndex(chars_file_path)

In [7]:
def get_input_layer(word_idx):
    x = torch.zeros(len(tokens)).float()
    x[word_idx] = 1.0
    return x

In [10]:
#意味ベクトルの読み込み
vec_of_char5 = torch.load("/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki/char2vec")
vec_of_char512 = torch.load("/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki/char2vec_512")


In [11]:
print(vec_of_char5.shape)
print(vec_of_char512.shape)


torch.Size([3565, 5])
torch.Size([3565, 512])


In [12]:
def char2vec(char,vec_of_char):
    idx = tokens.get_index(char)
    return vec_of_char[idx]

In [13]:
print(char2vec('知',vec_of_char512))
print(char2vec('知',vec_of_char5))


tensor([-7.4137e-01,  5.8941e-01,  5.9843e-01,  9.7174e-01, -1.7691e+00,
         1.7523e-01, -3.0012e-01, -8.1373e-05, -7.9159e-01, -5.7243e-01,
        -1.2546e-01,  4.1323e-01,  6.8645e-01,  3.7901e-01,  1.1630e+00,
         7.9563e-01, -1.0154e+00,  3.9286e-01, -1.0071e-01,  1.8784e-01,
         1.2511e+00,  3.8058e-01,  1.9712e-01, -4.9323e-02,  1.5533e+00,
        -1.8046e-01,  4.8649e-01, -1.4080e+00, -5.0718e-01, -7.3630e-01,
        -4.9081e-02,  8.3686e-02, -1.8827e-01, -4.3209e-01, -1.6373e+00,
         6.3175e-01,  1.5049e+00,  1.0622e+00,  1.2214e+00, -1.1460e+00,
        -3.0054e-01,  3.7541e-01, -4.6230e-01, -2.3891e-03,  2.1387e-01,
        -3.7908e-02,  6.1854e-01, -1.0906e-01, -1.4957e+00, -8.2266e-01,
         5.3021e-01,  1.0490e+00, -5.2519e-01, -1.1445e+00, -1.7280e+00,
        -1.9427e+00,  5.2561e-01,  1.9451e+00,  7.1250e-01,  1.9168e+00,
         1.7128e+00,  2.9312e-01,  9.8008e-01,  5.6424e-01, -5.7614e-01,
        -3.3921e-01,  1.5140e-01,  4.6575e-01,  1.5

In [19]:
def cos_sim(v1, v2):
    v1 = v1.detach().numpy()
    v2 = v2.detach().numpy()
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [20]:
def most_similar(char,vec_of_char,show_vec=False,with_cos_similar=False,num=10):
    vec = char2vec(char,vec_of_char)
    result = []
    cosign_similares = []
    for compare_vec in vec_of_char:
        cosign_similares.append(cos_sim(vec,compare_vec))
        sorted_similar_indexes = np.argsort(-np.array(cosign_similares))[:num]
    for index in sorted_similar_indexes:
        print(tokens.get_decoded_char(int(index)),end=': ')
        if show_vec:
            print(vec_of_char[index],end=' ')

        if with_cos_similar:
            print(cosign_similares[index])
        result.append(tokens.get_decoded_char(int(index)))
    print()
    return result

In [25]:
string = '力メラ１マイケなどの各種センサ'
for s in string:
    most_similar(s,vec_of_char512, with_cos_similar=False, show_vec=False, num=10)

力: tensor([-1.5734e+00,  9.5923e-01,  1.4440e+00, -2.3750e-01, -5.2843e-01,
         1.2765e+00, -7.8936e-01, -1.5490e-03, -3.8040e-01,  6.2786e-02,
         4.7382e-01, -6.2060e-01,  2.3103e-01,  3.4450e-02,  1.2891e+00,
        -8.4888e-01, -3.6693e-01, -1.6075e+00,  2.8954e-01, -1.1250e+00,
        -7.5051e-01,  3.5682e-01, -1.2978e+00,  1.1144e+00,  8.7789e-01,
         2.6199e-02, -2.9462e-01, -1.1567e+00,  1.2592e+00, -6.7361e-01,
         8.1687e-01, -3.6827e-02,  1.4333e+00,  6.8718e-02, -5.2137e-01,
         9.5171e-01,  2.3270e+00,  7.8027e-01,  1.0489e+00, -1.5589e-01,
         1.0904e-01,  8.7649e-01,  4.0113e-02,  6.7360e-01, -3.8086e-01,
        -8.7050e-01,  5.6503e-01,  2.5822e-01,  4.9896e-01, -4.9465e-01,
        -1.0380e+00,  2.2237e-01,  9.3810e-01, -6.1745e-01, -9.7828e-01,
         3.4774e-01, -1.2639e+00,  8.8641e-01,  1.3218e+00, -2.9718e-01,
         1.4693e-01,  3.7110e-01,  1.2330e+00,  1.7402e+00, -3.6497e-01,
         1.3139e+00, -1.0752e-01,  8.4604e-01,  

In [22]:
similares = most_similar('１',num=10,show_vec=True,with_cos_similar=True)

TypeError: most_similar() missing 1 required positional argument: 'vec_of_char'

In [ ]:
vec = char2vec('知')
vec1 = char2vec('１')
vec2 = char2vec('太')
vec3 = char2vec('筋')
vec4 = char2vec('注')
vec5 = char2vec('遍')
print(vec1,cos_sim(vec1,vec))
print(vec2,cos_sim(vec2,vec))
print(vec3,cos_sim(vec3,vec))
print(vec4,cos_sim(vec4,vec))
print(vec5,cos_sim(vec5,vec))




tensor([-1.6295, -0.1756,  0.6639,  1.9598, -5.1427], grad_fn=<SelectBackward>) 0.24026944
tensor([-0.6091, -1.7696, -1.0779,  2.9327, -1.4526], grad_fn=<SelectBackward>) 0.5552896
tensor([1.6082, 0.2388, 0.9839, 0.2555, 1.6631], grad_fn=<SelectBackward>) 0.24987587
tensor([ 0.0208, -3.3225, -1.0360,  0.4143, -0.4525], grad_fn=<SelectBackward>) -0.13658895
tensor([-0.1720,  0.0338,  2.2412,  0.1110,  1.7681], grad_fn=<SelectBackward>) 0.12535448


In [ ]:
def tmp(char,show_vec=False,with_cos_similar=False,num=10):
    vec = char2vec(char)
    result = []
    cosign_similares = []
    for compare_vec in w2:
        cosign_similares.append(cos_sim(vec,compare_vec))
        sorted_similar_indexes = np.argsort(np.array(cosign_similares))[500:num+500]
    for index in sorted_similar_indexes:
        print(tokens.get_decoded_char(int(index)),end=': ')
        print(w2[index],end=' ')

        if with_cos_similar:
            print(cosign_similares[index])
        result.append(tokens.get_decoded_char(int(index)))
    print()
    return result

In [ ]:
tmp('知',show_vec=True,with_cos_similar=True)

NameError: name 'w2' is not defined

In [ ]:
print(tokens.decoded_table())

In [ ]:
len_han_plus_zen = 0
han_arr = []
for char,i in tokens.table.items():
    # print(char.decode())
    len_han_plus_zen+=1
    han_arr.append(mojimoji.zen_to_han(char.decode()))
han_arr = np.unique(han_arr)
print(f'半角全角:{len_han_plus_zen}, 半角:{len(han_arr)}')


In [ ]:
for item in han_arr:
    print(item,end=' ')